In [ ]:
# Step 1: Install correct dependencies
print("Installing dependencies...")
!pip install -q torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q numpy==1.26.4 scipy==1.13.0 scikit-learn==1.4.2
!pip install -q matplotlib==3.8.0 seaborn==0.13.2 tqdm wandb
!pip install -q basicsr facexlib gfpgan umap-learn
!pip install -q --no-deps git+https://github.com/xinntao/Real-ESRGAN.git
print("Dependencies installed successfully.")

# Step 2: Imports
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader, Subset
from pathlib import Path
import numpy as np
import wandb, os, copy, math, glob, logging
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity as ssim_sk
from PIL import Image
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg19
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
import rasterio

# Step 3: Setup Devices and Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_count = torch.cuda.device_count()
print(f"Using device: {device}" + (f" with {gpu_count} GPUs (DataParallel)." if gpu_count > 1 else "."))

# (FIXED) Config updated for EuroSAT (10 classes) and high-performance training
config = {
    'project_name': 'sr-al-eurosat-definitive',
    'dataset_root': '/kaggle/input/eurosat/EuroSAT/', # Correct path for EuroSAT
    'sr_psnr_epochs': 10,  # Epochs for PSNR pre-training
    'sr_gan_epochs': 15,   # Epochs for GAN fine-tuning
    'batch_size': 16,
    'accumulation_steps': 2, # Effective batch size = 32
    'al_cycles': 5,
    'al_epochs': 15,
    'num_classes': 10,     # EuroSAT has 10 classes
    'lambda_perc': 1.0,    # Perceptual loss weight
    'eta_adv': 0.01,       # Adversarial loss weight
    'sr_g_lr': 1e-4, 'sr_d_lr': 1e-4, 'sr_psnr_lr': 2e-4,
    'sr_lr_milestones': [5, 10]
}

# Step 4: Login to WandB
try:
    wandb.login(key="5424a3d65aac1662f5be82d4439aaac35046689e")
    print("W&B login successful.")
except Exception as e:
    print(f"W&B login failed: {e}. Please log in manually.")
    wandb.login()

# (FIXED) Initialize W&B *without* a name to get random names
wandb.init(project=config['project_name'], config=config)
print(f"Setup complete. WandB run '{wandb.run.name}' started.")



Installing dependencies...
  Preparing metadata (setup.py) ... done
Dependencies installed successfully.


ValueError: module functions cannot set METH_CLASS or METH_STATIC